In [23]:
import pandas as pd
session = pd.read_csv('data/session_data_training_feb.csv', header = None)
session_columns = [
    'sessionid',
    'category',
    'imageurl',
    'createddate',
    'pagetitle',
    'pageurl',
    'userid',
    'fullurl',
    'providerid',
    'productid',
    'normalizedpageurl',
    'rawpageurl',
    'referrerurl',
    'rawreferrerurl',
    'utmsource',
    'utmmedium',
    'utmcontent',
    'utmcampaign',
    'utmterm',
    'ipaddress',
    'deviceid',
    'requesttype',
    'eventtype',
    'quantity',
    'price'
]
session.columns = session_columns
session.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247077 entries, 0 to 247076
Data columns (total 25 columns):
sessionid            247068 non-null object
category             190964 non-null object
imageurl             247071 non-null object
createddate          247077 non-null object
pagetitle            247077 non-null object
pageurl              247077 non-null object
userid               79801 non-null object
fullurl              191839 non-null object
providerid           247077 non-null object
productid            247077 non-null int64
normalizedpageurl    247077 non-null object
rawpageurl           247077 non-null object
referrerurl          184214 non-null object
rawreferrerurl       184214 non-null object
utmsource            3563 non-null object
utmmedium            3563 non-null object
utmcontent           0 non-null float64
utmcampaign          3563 non-null object
utmterm              0 non-null float64
ipaddress            247077 non-null object
deviceid             2470

In [29]:
%run Markovify.py

(247068, 14)


MemoryError: 

In [8]:
test = example.modeling_df

In [10]:
trunc = test[['sessionid','userid','createddate']]

In [19]:
agg = trunc.groupby(['sessionid','userid'],as_index = False).agg({'createddate': 'min'})

In [20]:
agg.head()

,sessionid,userid,createddate
0,0000ts732x9627fexoxjwxytf80gchp1zv5ptihvayjf0k...,Unknown,2017-02-18 16:08:50
1,001ki9xnoc3pw3y1xf2xcxzuxa4yi3hkicvswbxplrmbqs...,lemen.jennifer@gmail.com,2017-02-05 14:10:44
2,002jywn49yw5zx6qekrhsloeh4o6cro90x4smc8421e6ib...,jensmeets26@hotmail.com,2017-02-17 03:58:07
3,0056ihxkksrf1x55w5ssj25fij1dljumjvcchgvfipphiu...,Unknown,2017-02-05 02:28:42
4,005tkwryl1jvwunanno1ai2vfcp7141bajra013bv6iktg...,Unknown,2017-02-12 01:34:30


In [3]:
example.modeling_df['eventtype'].value_counts()

 1.0      98121
 3.0      23407
 104.0    10776
 4.0       6716
 7.0       6447
 6.0       4077
-1.0       2460
Name: eventtype, dtype: int64

In [4]:
98121./example.modeling_df.shape[0]

0.6455159074761191

In [5]:
example.rfc_score()

0.67256883666086231

In [7]:
for i in example.swym_x.columns:
    print i

quantity
price
elapsedtime
totalelapsedtime
Page View
Page View 1
Add to Cart
Add to Cart 1
Add to Wishlist
Add to Wishlist 1
Purchase
Purchase 1
Remove from Cart
Remove from Cart 1
Begin Checkout
Begin Checkout 1
Delete from Wishlist
Delete from Wishlist 1
Tuesday
Wednesday
Thursday
Friday
Saturday
Sunday
Hour 1
Hour 2
Hour 3
Hour 4
Hour 5
Hour 6
Hour 7
Hour 8
Hour 9
Hour 10
Hour 11
Hour 12
Hour 13
Hour 14
Hour 15
Hour 16
Hour 17
Hour 18
Hour 19
Hour 20
Hour 21
Hour 22
Hour 23
iPhone
Windows PC
Android phone
Mac
iPad
Linux PC
Android PC
Android tablet
Windows phone
Smartphone
Personal computer
Tablet
Mobile Browser
Browser
iOS
Android
Windows
OS X
Linux
android
armstrongmywire
cause
charlies
charliesproject
checkout
com
excite
facebook
gm
google
googlequicksearchbox
juno
leggings
live
lm
mail
messenger
msn
myshopify
outlook
pinterest
project
salesforce
search
shopify
shutterstock
slack
soundestlink
uk
webmail
webmail2
webmaila
www
yahoo
animal
awareness
baseball
camo
cause
collection


In [5]:
for i in range(5):
    test = Markovify(order = i+1)
    test.load_swym_data('data/session_data_training_feb.csv', 'data/devices_data_training_feb.csv')
    print test.rfc_score()

0.665173347097
0.673864709908


KeyboardInterrupt: 

In [9]:
test = Markovify(order = 5)
test.load_swym_data('data/session_data_sample_030113.csv', 'data/devices_data_sample_030113.csv')
print test.rfc_score()

0.718487398113


In [10]:
test = Markovify(order = 6)
test.load_swym_data('data/session_data_sample_030113.csv', 'data/devices_data_sample_030113.csv')
print test.rfc_score()

0.737955929034


In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np

example = Markovify()
example.load_swym_data('data/session_data_sample_030113.csv', 'data/devices_data_sample_030113.csv')

category_gs = example.modeling_df['category']
y = example.swym_y

In [49]:
tfidf = TfidfVectorizer(stop_words = 'english')
x = tfidf.fit_transform(category_gs)
rfc = RandomForestClassifier()
print x.shape, np.mean(cross_val_score(rfc,x,y,cv=5))

(67737, 4342) 0.707162280855


In [50]:
max_features = np.linspace(100,5000,50).astype(int)

for i in max_features:
    tfidf = TfidfVectorizer(stop_words = 'english', max_features = i)
    x = tfidf.fit_transform(category_gs)
    rfc = RandomForestClassifier()
    print i, np.mean(cross_val_score(rfc,x,y,cv=5))


100 0.714927343392
200 0.712786362406
300 0.710690132676
400 0.710114539231
500 0.708844886441
600 0.708136535607
700 0.707516493874
800 0.707930078808
900 0.706748758074
1000 0.707428097593
1100 0.70804794639
1200 0.70790043864
1300 0.706675248319
1400 0.706187784182
1500 0.707103294773
1600 0.707236198218
1700 0.706261873741
1800 0.706335546968
1900 0.706867049562
2000 0.709273365568
2100 0.706572032982
2200 0.708623812317
2300 0.705287787904
2400 0.705715628368
2500 0.706070007065
2600 0.706793400329
2700 0.706822735344
2800 0.706453707694
2900 0.706055280723
3000 0.707442631026
3100 0.706911307113
3200 0.706852367893
3300 0.707797023882
3400 0.706350529422
3500 0.706512985858
3600 0.706660465275
3700 0.707117865282
3800 0.707664134602
3900 0.707147421532
4000 0.706379672631
4100 0.706881756329
4200 0.705656441728
4300 0.706807920689
4400 0.708121623973
4500 0.708165893534
4600 0.706099629797
4700 0.707059105877
4800 0.706926169725
4900 0.706483100471
5000 0.706793205252


In [52]:
tfidf = TfidfVectorizer(stop_words = 'english', max_features=10)
x = tfidf.fit_transform(category_gs)
rfc = RandomForestClassifier()
print x.shape, np.mean(cross_val_score(rfc,x,y,cv=5))

(67737, 10) 0.715753933438


In [53]:
tfidf.get_feature_names()

[u'arrivals',
 u'best',
 u'business',
 u'cases',
 u'inches',
 u'new',
 u'prints',
 u'products',
 u'tools',
 u'women']